# Bivariate Choropleth
This notebook will outline the procedure to construct a bivarate choropleth map in Python, using NYC PLUTO data.

In [84]:
# import necessary packages
import pandas as pd
import numpy as np
import os
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from census import Census
from dotenv import load_dotenv
load_dotenv()


# set API key
CENSUS_API = os.getenv("CENSUS_API")
c = Census(CENSUS_API, year=2019)

# surpress warnings
import warnings
warnings.filterwarnings('ignore')

## Load data

First we need to download the data, and next the shapefile.

### Download data

In [156]:
# variables
variables=('NAME',"B03002_001E","B03002_003E","B03002_004E","B03002_012E","B03002_006E","B17001_002E")

# download census data
ny_tract=c.acs5.state_county_tract(fields=variables,
                           state_fips=36,
                           county_fips="*",
                           tract="*")

In [157]:
# convert to df 
df_ny_tract=pd.DataFrame(ny_tract)
# create county fips list
nyc_county_fips = ["005","047","061","081","085"]
# filter for nyc
df=df_ny_tract[df_ny_tract["county"].isin(nyc_county_fips)]

In [158]:
def perc_rate(df, pop_col, other_col_list, new_col_list):
    """
    Takes in df, pop col, other col list, and new name  list
    drops original columns and renames pop column
    """
    # loop through other col list and name list
    for col, name in zip(other_col_list, new_col_list):
        df['temp'] = round(df[col] / df[pop_col] * 100,1)
        df[name] = df['temp']

    # rename pop col
    df['pop_count'] = df[pop_col]

    # drop the original columns for the population counts and keep only the new columns
    df.drop(columns=other_col_list + ['temp',pop_col], inplace=True)

In [159]:
col_list = ["B03002_003E","B03002_004E","B03002_012E","B03002_006E","B17001_002E"]
name_list = ['white_pct', 'black_pct', 'hispanic_pct', 'asian_pct', 'poverty_rate']

perc_rate(df, "B03002_001E", col_list, name_list)

In [160]:
df.head()

,NAME,state,county,tract,white_pct,black_pct,hispanic_pct,asian_pct,poverty_rate,pop_count
8,"Census Tract 361, Queens County, New York",36,081,036100,3.3,16.9,64.7,14.4,13.0,2238.0
9,"Census Tract 363, Queens County, New York",36,081,036300,3.7,14.9,72.8,6.5,20.3,1771.0
10,"Census Tract 371, Queens County, New York",36,081,037100,1.4,23.1,54.2,15.0,11.8,1335.0
11,"Census Tract 377, Queens County, New York",36,081,037700,0.5,5.3,90.1,4.2,12.1,3620.0
12,"Census Tract 379, Queens County, New York",36,081,037900,0.9,4.0,87.0,7.8,16.3,6851.0


Let's look at column info

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2167 entries, 8 to 4887
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   NAME          2167 non-null   object 
 1   state         2167 non-null   object 
 2   county        2167 non-null   object 
 3   tract         2167 non-null   object 
 4   poverty_rate  2124 non-null   float64
 5   white_pct     2124 non-null   float64
 6   black_pct     2124 non-null   float64
 7   hispanic_pct  2124 non-null   float64
 8   asian_pct     2124 non-null   float64
dtypes: float64(5), object(4)
memory usage: 169.3+ KB


In [96]:
df["county"].unique()

array(['081', '005', '085', '061', '047'], dtype=object)

### Download shapefile

Ok, now let's load in the shapefile from [NYC Open Data](https://data.cityofnewyork.us/City-Government/2010-Census-Tracts/fxpq-c8ku)

In [88]:
# load shpfile as gdf
gdf=gpd.read_file("https://data.cityofnewyork.us/api/geospatial/fxpq-c8ku?method=export&format=Shapefile")

# preview
gdf.head()

,boro_code,boro_ct201,boro_name,cdeligibil,ct2010,ctlabel,ntacode,ntaname,puma,shape_area,shape_leng,geometry
0,5,5000900,Staten Island,E,000900,9,SI22,West New Brighton-New Brighton-St. George,3903,2.497010e+06,7729.016794,"POLYGON ((-74.07921 40.64343, -74.07914 40.643..."
1,1,1009800,Manhattan,I,009800,98,MN19,Turtle Bay-East Midtown,3808,1.906016e+06,5534.200308,"POLYGON ((-73.96433 40.75638, -73.96479 40.755..."
2,1,1010200,Manhattan,I,010200,102,MN17,Midtown-Midtown South,3807,1.860993e+06,5687.802439,"POLYGON ((-73.97124 40.76094, -73.97170 40.760..."
3,1,1010400,Manhattan,I,010400,104,MN17,Midtown-Midtown South,3807,1.864600e+06,5693.036367,"POLYGON ((-73.97446 40.76229, -73.97491 40.761..."
4,1,1011300,Manhattan,I,011300,113,MN17,Midtown-Midtown South,3807,1.890907e+06,5699.860640,"POLYGON ((-73.98412 40.75484, -73.98460 40.754..."


In [87]:
# get info
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2165 entries, 0 to 2164
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   boro_code   2165 non-null   object  
 1   boro_ct201  2165 non-null   object  
 2   boro_name   2165 non-null   object  
 3   cdeligibil  2164 non-null   object  
 4   ct2010      2165 non-null   object  
 5   ctlabel     2165 non-null   object  
 6   ntacode     2165 non-null   object  
 7   ntaname     2165 non-null   object  
 8   puma        2165 non-null   object  
 9   shape_area  2165 non-null   float64 
 10  shape_leng  2165 non-null   float64 
 11  geometry    2165 non-null   geometry
dtypes: float64(2), geometry(1), object(9)
memory usage: 203.1+ KB


In [90]:
gdf[gdf['ct2010'] == '036100']

,boro_code,boro_ct201,boro_name,cdeligibil,ct2010,ctlabel,ntacode,ntaname,puma,shape_area,shape_leng,geometry
687,3,3036100,Brooklyn,E,036100,361,BK79,Ocean Hill,4007,1.524421e+06,6064.700132,"POLYGON ((-73.91487 40.67053, -73.91553 40.670..."
703,2,2036100,Bronx,E,036100,361,BX17,East Tremont,3705,1.760194e+06,5701.061491,"POLYGON ((-73.87687 40.84232, -73.87703 40.841..."
1225,4,4036100,Queens,E,036100,361,QN27,East Elmhurst,4102,1.693935e+06,7450.199750,"POLYGON ((-73.87135 40.76233, -73.87134 40.762..."


Ok, it looks like we can join on `boro_ct2010` from **gdf**, but will have to create the `boro_code` column in **df** in order to make this possible. So let's create a feature that maps the correct boro code in **df** then concatenate it with `tract` column to match `boro_ct2010`.

- Manhattan = 1 = 061
- Bronx = 2 = 005
- Brooklyn = 3 = 047
- Queens = 4 = 081
- Staten Island = 5 = 085

In [97]:
# set variables
col = 'county'
conditions = [df[col] == "061",df[col] == "005",df[col] == "047",df[col] == "081",df[col] == "085"]
choices = ['1','2','3','4','5']

# compute conditional
df['boro_code'] = np.select(conditions, choices)

In [98]:
df.head()

,NAME,state,county,tract,poverty_rate,white_pct,black_pct,hispanic_pct,asian_pct,boro_code
8,"Census Tract 361, Queens County, New York",36,081,036100,12.957998,3.261841,16.890080,64.745308,14.432529,4
9,"Census Tract 363, Queens County, New York",36,081,036300,20.271033,3.726708,14.850367,72.840203,6.493506,4
10,"Census Tract 371, Queens County, New York",36,081,037100,11.835206,1.423221,23.146067,54.232210,14.981273,4
11,"Census Tract 377, Queens County, New York",36,081,037700,12.127072,0.469613,5.276243,90.055249,4.198895,4
12,"Census Tract 379, Queens County, New York",36,081,037900,16.274996,0.919574,3.955627,87.023792,7.809079,4


Ok, now we can join

### Join

In [100]:
# join gdf with df
df_merged=pd.merge(gdf, df)

Now that I have the data and shapefile, next step is to create basic choropleth map off with to build.